## Big Bang台词分析
- 生活大爆炸
- 数据获取[地址](https://github.com/skashyap7/TBBTCorpus)
- ![](https://pmcvariety.files.wordpress.com/2019/05/the-big-bang-theory-artisans.jpg?w=1000&h=563&crop=1)

## 数据准备
- 下载数据，将json格式加载进来
- 发现一共202篇台词

In [149]:
import json
import sys
# 数据来自：https://github.com/skashyap7/TBBTCorpus
data_file = 'C:/Users/wqw/Desktop/corpus.json'
for line in open(data_file).readlines():
    data_raw = json.loads(line)
    print(len(data_raw.keys()), data_raw.keys())

202 dict_keys(['8_17', '7_23', '7_10', '3_7', '4_2', '8_1', '5_12', '7_6', '9_2', '3_2', '3_11', '2_19', '1_4', '2_2', '6_21', '6_22', '5_23', '4_18', '2_20', '4_14', '6_3', '4_12', '8_20', '5_20', '8_14', '5_17', '2_13', '3_18', '7_5', '5_13', '5_7', '2_6', '7_12', '1_12', '4_22', '5_4', '6_1', '4_21', '3_1', '9_1', '6_12', '6_6', '7_24', '4_24', '1_10', '1_6', '2_1', '2_23', '3_21', '2_11', '3_13', '1_3', '6_14', '5_15', '2_16', '9_19', '1_17', '5_18', '1_13', '6_19', '2_17', '9_3', '8_11', '8_21', '6_13', '7_18', '1_9', '5_2', '3_5', '5_11', '2_9', '2_4', '1_11', '9_6', '8_9', '8_22', '4_6', '5_16', '5_19', '4_13', '1_15', '4_9', '3_3', '5_8', '1_1', '4_7', '7_14', '3_14', '4_1', '8_6', '6_7', '5_6', '7_15', '3_19', '8_4', '1_7', '3_22', '7_13', '6_15', '5_21', '7_17', '2_5', '8_16', '2_21', '9_9', '7_2', '6_9', '3_9', '4_11', '8_12', '9_15', '6_4', '5_22', '1_5', '8_8', '2_14', '4_3', '9_17', '8_7', '6_2', '9_23', '2_7', '6_5', '2_15', '6_16', '8_19', '2_12', '9_4', '6_10', '7_11',

## 计算台词数目
- 每个场景下，每个人物的台词数据处理
- 统计台词数目

In [94]:
import sys
#data_raw['8_9']
# 每个场景、每个人
sen_dict = {}
cnt = 0
for k, v in data_raw.items():
    cnt += 1
    if cnt > 2:
        #break
        pass
    # 读取每篇文章
    first_line = False
    for line in v.strip().split('\n'):
        if not first_line:
            scene = line.strip('.').split(':')[1].strip()
            first_line = True
            continue
        arr = [i.strip() for i in line.split(':')]
        if len(arr) !=2:
            print(sys.stderr, '异常数据:%s'%(line))
            continue
        person, sen = arr
        # 数据清洗，Sheldon (on video)， Like this
        if person.find(' (') != -1:
            person = person.split(' (')[0]
        if not scene in sen_dict:
            sen_dict[scene] = {}
        # 聚合每个场景下的同一个人
        if not person in sen_dict[scene]:
            sen_dict[scene][person] = []
        sen_dict[scene][person].append(sen)
    print('%s\t%s'%(title, line))
    print(sen_dict)

<ipykernel.iostream.OutStream object at 0x0000016F9F300B00> 异常数据:Credits sequence.
 The upstairs landing of Mrs Wolowitz’s house	Like this:Like Loading...
{'The stairwell': {'Leonard': ['I was unstoppable. I mean, I was, I was on fire. It was like my mind and my body were totally connected, like, like athletes must feel when they’re in the zone.', 'Admit it, you’re a little turned on.', 'Why not?', 'Hey.', 'Okay.', 'Okay.', 'Well, we’re, we’re very happy for you.', 'Why a turtle?', 'Well, congratulations. Who would’ve thought you two would be the first in our group to start a family?', 'Hey, hey. Listen, I, I have a little confession to make.', 'No, I got you something for Valentine’s Day, and I was too embarrassed to give it to you.', 'Well, ’cause I got it at the dirty store.', 'In sunglasses and a hat after I parked two blocks away.', 'Yeah?', 'Okay, uh, well, I hope it’s fun. I mean, it, it comes with paints, and, and it’s kind of creative and artistic.', 'No, no. We cover ourselve

IndexError: list index out of range

## NLP提取关键词
- 句子分词，提取关键词
- 看看每个人物都爱说啥

In [95]:
# 每个场景、每个人分词
import jieba
from jieba import analyse
word_dict = {}
for s in sen_dict:
    if s not in word_dict:
        word_dict[s] = {}
    for p in sen_dict[s]:
        if p not in word_dict[s]:
            word_dict[s][p] = {}
        # 每个场景每个人说过的所有话，列表
        speak_list = sen_dict[s][p]
        for w in speak_list:
            # 提取关键词
            #seg_list = jieba.cut(text)
            #res = analyse.TFIDF(w) # tf-idf提取关键词
            res = analyse.extract_tags(w)
            #res = analyse.textrank(w) # text-rank提取关键词
            for word in res:
                word_dict[s][p][word] = word_dict[s][p].get(word, 0) + 1
print(json.dumps(word_dict, ensure_ascii=False, indent=2))

{
  "The stairwell": {
    "Leonard": {
      "was": 3,
      "like": 3,
      "my": 1,
      "unstoppable": 1,
      "mean": 4,
      "fire": 2,
      "mind": 1,
      "body": 2,
      "were": 2,
      "totally": 1,
      "connected": 1,
      "athletes": 1,
      "must": 1,
      "feel": 1,
      "when": 1,
      "they": 1,
      "re": 4,
      "zone": 2,
      "Admit": 1,
      "little": 3,
      "turned": 1,
      "Why": 2,
      "Hey": 2,
      "Okay": 4,
      "Well": 4,
      "very": 1,
      "happy": 1,
      "turtle": 1,
      "would": 1,
      "congratulations": 1,
      "Who": 1,
      "ve": 1,
      "thought": 2,
      "two": 2,
      "first": 1,
      "our": 3,
      "group": 1,
      "start": 1,
      "family": 1,
      "hey": 1,
      "Listen": 1,
      "confession": 1,
      "make": 1,
      "No": 2,
      "got": 2,
      "something": 1,
      "Valentine": 1,
      "Day": 1,
      "too": 1,
      "embarrassed": 1,
      "give": 1,
      "cause": 1,
      "dirty": 1,
   

### 测试
- 查看效果，指定任意场景、人物，显示其常见词语

In [96]:
scene = 'The stairwell'
person = 'Leonard'
#word_dict.values()
#word_dict[scene][person]
sen_dict[scene][person]

['I was unstoppable. I mean, I was, I was on fire. It was like my mind and my body were totally connected, like, like athletes must feel when they’re in the zone.',
 'Admit it, you’re a little turned on.',
 'Why not?',
 'Hey.',
 'Okay.',
 'Okay.',
 'Well, we’re, we’re very happy for you.',
 'Why a turtle?',
 'Well, congratulations. Who would’ve thought you two would be the first in our group to start a family?',
 'Hey, hey. Listen, I, I have a little confession to make.',
 'No, I got you something for Valentine’s Day, and I was too embarrassed to give it to you.',
 'Well, ’cause I got it at the dirty store.',
 'In sunglasses and a hat after I parked two blocks away.',
 'Yeah?',
 'Okay, uh, well, I hope it’s fun. I mean, it, it comes with paints, and, and it’s kind of creative and artistic.',
 'No, no. We cover ourselves in body paint, and then, then we get on this big canvas and do our thing.',
 'You’re making fun of me. Forget it.',
 'Really?',
 'Okay, but it’s a drive, the one I went

## 人物台词可视化

In [148]:
import pyecharts as pe

# scene = 'The stairwell'
# person = 'Leonard'
page = pe.Page('老友记词云可视化')
person_count = {}
for scene in word_dict:
    for person in word_dict[scene]:
        name_list = word_dict[scene][person].keys()
        freq_list = word_dict[scene][person].values()
        chart = pe.WordCloud('Scene: %s'%(scene), subtitle='Person: %s'%(person), title_pos='center')
        chart.add("", name_list, freq_list, word_size_range=[30, 100], rotate_step=66)
        page.add(chart)
        # 统计台词数
        person_count[person] = person_count.get(person, 0) + len(sen_dict[scene][person])
# 添加台词数分布
# bar = pe.Bar()
# name_list = person_count.keys()
# val_list = person_count.values()
# bar.add("", name_list, val_list)
# page.add(bar)
page.render('vis.html')
page

[<pyecharts.charts.wordcloud.WordCloud at 0x16fbb3926a0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb30fc88>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb30f2e8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392470>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392630>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392400>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392978>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3927b8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392898>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392780>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392908>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392668>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3928d0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392240>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3929e8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3927f0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392048>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa6ff4668>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa72db7f0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa6ff4e80>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392080>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392b38>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392ac8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392a20>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392be0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392c50>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3926d8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3929b0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392cc0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392d68>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb322ef0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb3220b8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbb392b00>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faadac0f0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa724e6d8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faad0b5f8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba936f60>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa6fd5080>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16f9f58e080>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fbadf6d68>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7263b00>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa6fd5f98>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faae03358>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa70015f8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7001470>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faae032e8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faae03320>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faae01e80>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa459bbe0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7257b00>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa6ffcac8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16faae03898>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7003940>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7003860>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7003438>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fa7003fd0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba917cc0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba917f28>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba917fd0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba9174e0>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba917dd8>,
 <pyecharts.charts.wordcloud.WordCloud at 0x16fba917710>]

In [133]:
'The stairwell' in list(word_dict.keys())
'The stairwell' in word_dict.keys()

True

## 热力图可视化

In [144]:
import pyecharts as pe
from numpy import random

# scene = 'The stairwell'
# person = 'Leonard'
# page = pe.Page('老友记词云可视化') 
p_list = []
x_axis = [] # 所有场景
for i in word_dict:
    x_axis.append(i)
    p_list.extend(word_dict[i].keys())
print(p_list)
x_len = len(x_axis)
print('x: ',x_axis)
y_axis = list(set(p_list)) # 所有人物
y_len = len(y_axis)
print('y: ',y_axis)
print(type(x_axis), type(y_axis))
data = []
for i in range(x_len):
    val = 0
    #print('xx===>',x_axis[i], word_dict)
    if x_axis[i] not in word_dict:
        print('x===>',x_axis[i], word_dict.keys())
        val = 0
    #continue
    for j in range(y_len):
        if x_axis[i] not in word_dict:
            print('--->100', x_axis[i], word_dict.keys())
            pass
        elif y_axis[j] not in word_dict[x_axis[i]]:
            #print('y===>',x_axis[i], y_axis[i], word_dict[x_axis[i]].keys())
            val = 0
        else:
            val = len(word_dict[x_axis[i]][y_axis[j]]) # 单词频次
            #val = len(sen_dict[x_axis[i]][y_axis[j]]) # 说话频率
        data.append([j, i, val])
print(data)
#data = [[i, j, random.randint(0, 50)] for i in range(x_len) for j in range(y_len)]
heatmap = pe.HeatMap()
heatmap.add("Scene-Person Heatmap", y_axis, x_axis, data, is_visualmap=True,
            visual_text_color="#000", visual_orient='horizontal')
heatmap.render('heatmap_freq.html')
heatmap

['Leonard', 'Penny', 'Sheldon', 'Amy', 'Together', 'Scene', 'Raj', 'Emily', 'Howard', 'Bernadette', 'Like this', 'Bernadette', 'Howard', 'Raj', 'Mrs Wolowitz', 'Scene', 'Leonard', 'Amy', 'Sheldon', 'Penny', 'Emily', 'Man', 'Director', 'Wil Wheaton', 'Wil', 'Foreign Nurse', 'Like this', 'Ira Flatow', 'Sheldon', 'Ira', 'Scene', 'Amy', 'Leonard', 'Penny', 'Bernadette', 'Howard', 'Wil Wheaton', 'Wil', 'Raj', 'Bermadette', 'Barry', 'Like this', 'Penny', 'Sheldon', 'Leonard', 'Scene', 'Raj', 'Howard', 'Stuart', 'Mrs Koothrappali', 'Dr Koothrappali', 'Howard’s Mother', 'Like this', 'Leonard', 'Sheldon', 'Scene', 'Howard', 'Penny', 'Sheldon-bot', 'Raj', 'Steve Wozniak', 'Like this']
x:  ['The stairwell', 'The upstairs landing of Mrs Wolowitz’s house', 'A radio studio', 'The apartment', 'The apartment. Sheldon has a series of whiteboards across the room']
y:  ['Mrs Koothrappali', 'Like this', 'Scene', 'Penny', 'Mrs Wolowitz', 'Howard', 'Sheldon-bot', 'Howard’s Mother', 'Foreign Nurse', 'Wil Whe

In [93]:
from numpy import random
from pyecharts import HeatMap

x_axis = ["12a", "1a", "2a", "3a", "4a", "5a", "6a", "7a", "8a", "9a", "10a", "11a",
          "12p", "1p", "2p", "3p", "4p", "5p", "6p", "7p", "8p", "9p", "10p", "11p"]
y_aixs = ["Saturday", "Friday", "Thursday", "Wednesday", "Tuesday", "Monday", "Sunday"]
data = [[i, j, random.randint(0, 50)] for i in range(24) for j in range(7)]
heatmap = HeatMap()
heatmap.add("热力图直角坐标系", x_axis, y_aixs, data, is_visualmap=True,
            visual_text_color="#000", visual_orient='horizontal')

heatmap

In [60]:
from pyecharts import Parallel

schema = ["data", "AQI", "PM2.5", "PM10", "CO", "NO2"]
data = [
        [1, 91, 45, 125, 0.82, 34],
        [2, 65, 27, 78, 0.86, 45,],
        [3, 83, 60, 84, 1.09, 73],
        [4, 109, 81, 121, 1.28, 68],
        [5, 106, 77, 114, 1.07, 55],
        [6, 109, 81, 121, 1.28, 68],
        [7, 106, 77, 114, 1.07, 55],
        [8, 89, 65, 78, 0.86, 51, 26],
        [9, 53, 33, 47, 0.64, 50, 17],
        [10, 80, 55, 80, 1.01, 75, 24],
        [11, 117, 81, 124, 1.03, 45]
]
parallel = Parallel("平行坐标系-默认指示器")
parallel.config(schema) 
parallel.add("parallel", data, is_random=True)

parallel